Aprendizaje Automatico. Actividad 2

Maria Araceli Díaz Cumplido

# Procesamiento de imágenes

Mejora el código de keras_model.py visto en la clase sobre Deep Learning para conseguir el mejor resultado posible en el problema de reconocimiento de dígitos manuscritos.

Entrega este cuaderno con el código modificado y ejecutado, indicando la accuracy del modelo y otras métricas de clasificación que consideres interesantes.

In [4]:
import numpy as np
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow import keras


In [6]:
EPOCHS = 10


def make_model(output_dim, dense_layer_sizes=[40], filters=20, kernel_size=(3, 3),pool_size=(2, 2), dropout=0.25):
    
    model = Sequential()
    # Add pair of convolutional-pooling hidden layers
    model.add(Conv2D(filters, kernel_size, activation='relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    # Add the flatten layer before the dense ones
    model.add(Flatten())
    # Add the dense hidden layers
    for layer_size in dense_layer_sizes:
        model.add(Dense(layer_size, activation='relu'))
    # Add a dropout layer to avoid overfitting
    model.add(Dropout(dropout))
    # Add the output layer
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])
    return model


def main():

    # Train
    data = np.loadtxt('/content/sample_data/data/train.csv', dtype='f8', delimiter=',', skiprows=1)
    X, y = data[:, 1:], data[:, 0]
    input_shape = (28, 28, 1)
    X = X.reshape(X.shape[0], *input_shape).astype('float32') / 255
    num_classes = len(np.unique(y))
    classifier = KerasClassifier(make_model, output_dim=num_classes,epochs=EPOCHS)
    search_space = {'dense_layer_sizes': [[32], [32, 32]], 'filters': [8, 16]}
    estimator = GridSearchCV(classifier, search_space, cv=3)
    estimator.fit(X, y)
    print('Best params: ' + str(estimator.best_params_))
    print('Best score: ' + str(estimator.best_score_))

    # Test
    X_test = np.loadtxt('/content/sample_data/data/test.csv', dtype='f8', delimiter=',', skiprows=1)
    X_test = X_test.reshape(X_test.shape[0], *input_shape).astype('float32') / 255
    ids = np.arange(1, len(X_test) + 1)
    predictions = estimator.predict(X_test)
    np.savetxt('/content/sample_data/data/keras_submission.csv', np.transpose([ids, predictions]),fmt=('%d', '%d'), delimiter=',', header='ImageId,Label',comments='')

    
if __name__ == '__main__':
    main()

<ipython-input-6-c49b62c2fa21>:32: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(make_model, output_dim=num_classes,epochs=EPOCHS)


Epoch 1/10
875/875 [==============================] - 11s 11ms/step - loss: 0.5363 - accuracy: 0.8341
Epoch 2/10
875/875 [==============================] - 9s 10ms/step - loss: 0.2584 - accuracy: 0.9222
Epoch 3/10
875/875 [==============================] - 9s 11ms/step - loss: 0.1942 - accuracy: 0.9419
Epoch 4/10
875/875 [==============================] - 10s 11ms/step - loss: 0.1604 - accuracy: 0.9513
Epoch 5/10
875/875 [==============================] - 8s 10ms/step - loss: 0.1397 - accuracy: 0.9574
Epoch 6/10
875/875 [==============================] - 11s 12ms/step - loss: 0.1240 - accuracy: 0.9615
Epoch 7/10
875/875 [==============================] - 10s 11ms/step - loss: 0.1103 - accuracy: 0.9664
Epoch 8/10
875/875 [==============================] - 10s 11ms/step - loss: 0.0995 - accuracy: 0.9691
Epoch 9/10
875/875 [==============================] - 8s 9ms/step - loss: 0.0904 - accuracy: 0.9717
Epoch 10/10
438/438 [==============================] - 2s 5ms/step - loss: 0.0757 - acc

Hay varias formas de mejorar este modelo como:

- Aumentar el número de capas convolucionales.
- Agregar capas de normalización.
- Ajustar la tasa de aprendizaje.
- Cambiar la arquitectura del modelo.
- Aumentar el número de épocas de entrenamiento

Despues de intentar varios codigos diferentes y todos ellos tardar muchisimo tiempo en generarse, he decidio quedarme con este en el que aumento las capas convolucionales a 2, agrego 2 capas densas y subo las epocas a 30. En el codigo propuesto se utiliza una sola capa convolucional seguida de una capa de aplanamiento y capas densas, mientras que mi propuesta se utiliza múltiples capas convolucionales y de agrupación seguidas de una capa de aplanamiento y capas densas.

En lugar de utilizar KerasClassifier y GridSearchCV, se utiliza una estrategia de validación cruzada (StratifiedKFold) para evaluar el modelo y compilarlo con una función de pérdida de entropía cruzada dispersa. El entrenamiento se realiza utilizando bucles de validación cruzada, y después de entrenar el modelo, se realiza una predicción en el conjunto de prueba y se guarda tambien en un archivo CSV llamado "keras_submission_capas_convolucionales.csv".

Al ejecutar el codigo  da un Average mayor que el porcionado en la practica 0.9792 vs 0.9939 por lo que considero que es significativamente mejor. 

In [7]:
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import numpy as np
import tensorflow as tf


# Establece la sesión de TensorFlow
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.allow_soft_placement = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

EPOCHS = 30
BATCH_SIZE = 64
LEARNING_RATE = 0.001

def make_model(input_shape, num_classes, filters, kernel_size, pool_size, hidden_units):
    model = Sequential()
    for i, num_filter in enumerate(filters):
        if i == 0:
            model.add(Conv2D(num_filter, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
        else:
            model.add(Conv2D(num_filter, kernel_size=kernel_size, activation='relu'))
        model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Flatten())
    for units in hidden_units:
        model.add(Dense(units, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def main():
    data = np.loadtxt('/content/sample_data/data/train.csv', dtype='f8', delimiter=',', skiprows=1)
    X, y = data[:, 1:], data[:, 0]
    input_shape = (28, 28, 1)
    
    num_classes = len(np.unique(y))
    model = make_model(input_shape=input_shape, num_classes=num_classes, filters=[20, 40],
                       kernel_size=(3, 3), pool_size=(2, 2), hidden_units=[32])

    # Entreno y evalúo el modelo usando validación cruzada
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        X_train = X_train.reshape(X_train.shape[0], *input_shape).astype('float32') / 255
        X_val = X_val.reshape(X_val.shape[0], *input_shape).astype('float32') / 255

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=EPOCHS, verbose=0)

        _, accuracy = model.evaluate(X_val, y_val, verbose=0)
        scores.append(accuracy)

    print('Average score: ' + str(np.mean(scores)))

    X_test = np.loadtxt('/content/sample_data/data/test.csv', dtype='f8', delimiter=',', skiprows=1)
    X_test = X_test.reshape(X_test.shape[0], *input_shape).astype('float32') / 255

    model.fit(X.reshape(X.shape[0], *input_shape).astype('float32') / 255, y, epochs=EPOCHS, verbose=0)

    predictions = model.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)

    # Guardar las predicciones en un archivo CSV
    output = np.column_stack((np.arange(1, len(predicted_labels) + 1), predicted_labels))
    np.savetxt('/content/sample_data/data/keras_submission_capas_convolucionales.csv', output, header='ImageId,Label', delimiter=',', fmt='%d', comments='')

if __name__ == '__main__':
    main()



Average score: 0.9939761956532797
875/875 [==============================] - 6s 6ms/step
